In [ ]:
import numpy as np
import pandas as pd

from numba import njit


In [ ]:
from oscb import *

# customize Bokeh plotting in the Jupyter Notebook environment
setup_notebook()

In [ ]:
# use NLTK to get corpus data

import nltk

nltk.download('brown')

from nltk.corpus import brown


In [ ]:
# Seed the PRNG
random_seed(12345)

In [ ]:
# create an lnet, it initially has no synaptic connection at all
# with each letter having 10 columns, 5x that number of total columns,
# and 100 cells per column, there are 130K neurons total.
#
# if all neurons are fully connected, there'll be 16.9B synapses/parameters.
#
# a LetterNet will hard-cap the number of synapses, 1M by default,
# it'll drop weakest synapses out, once the total number reaches 1M, a
# LOAD_FACTOR=0.8 configues that each compaction (synapse dropout) should
# retain 0.8M synapses.
#
lnet = LetterNet(
    MAX_SYNAPSES=1_000_000,  # max number of synapses, one million by default
    N_COLS_PER_LETTER=10,  # distributedness of letter SDR
    SPARSE_FACTOR=5,  # sparseness of letter SDR
    N_CELLS_PER_COL=100,  # per mini-column capacity
)

In [ ]:
# create an lsim, it initially have all neurons rest at the specified voltage,
# and can later carry out simulations batch after batch.
#
# it maintains a Bokeh figure which can be shown in any notebook cell, which can
# be updated per later batches of simulation
#
lsim = LetterNetSim(
    lnet,

    # global scaling factor, to accommodate a unit synaptic efficacy value of 1.0
    # roughly this specifies that:
    #   how many presynaptic spikes is enough to trigger a postsynaptic spike,
    #   when each synapse has a unit efficacy value of 1.0
    SYNAP_FACTOR=10,

    # reset voltage, negative to enable refractory period
    VOLT_RESET=-0.1,
    # membrane time constant
    τ_m=10,

    # fire plot params
    plot_width=800,
    plot_height=600,
    plot_n_steps=300,
    fire_dots_glyph="square",
    fire_dots_alpha=0.01,
    fire_dots_size=3,
    fire_dots_color="#0000FF",
)

In [ ]:
# the simulation has no batch performed at this moment,
# we can show an empty plot here, and save a handle to update
# it later
figh = show(lsim.fig, notebook_handle=True)

In [ ]:
# the underlying lnet has NO synaptic connection at all at this moment,
# we simulate a batch, and will only see prompted spikes, in the above plot,
# once updated
lsim.simulate(
    10, # n steps

    'xxx',  # a single word or list of words to prompt
    prompt_blur=0.8,  # reduce voltage of other cells than the prompted letter
)
push_notebook(handle=figh)  # Update the plot

In [ ]:
# one more batch with some different prompt
lsim.simulate(
    20, # n steps

    'jump',  # a single word or list of words to prompt
    prompt_blur=0.7,  # reduce voltage of other cells than the prompted letter
)
push_notebook(handle=figh)  # Update the plot

Note you can visually decipher the spikes in the plot, 
with column spans along the y-axis, and time-steps along the x-axis.

Where each time step has 0.8 of its x-span to render cells per minicolumn.

In [ ]:
# make a unit synaptic connection, from a former letter, to the immediate letter following it
lnet.learn_words_as_sequence(brown.words(
    # select part of the corpus if desirable, or
    # comment following line out and the full corpus is used
#     categories=[ 'news', 'reviews', ],
))

# note: `lnet.learn_words( ... )` will work similarly,
# it just doesn't connect the last letter of the former word,
# to first letter of the following word.

In [ ]:
# get a DataFrame about all excitatory synapses so far
excit_df = lnet.excitatory_synapses()
excit_df

In [ ]:
# statistics of individual synapses
excit_df.efficacy.describe()

In [ ]:
# statistics of all incoming synapses per individual postsynaptic neurons
excit_df.groupby(['to_column', 'to_cell'])['efficacy'].sum().describe()

In [ ]:
# now simulate one more batch with those synaptic connections
# scroll back to see the spike plot updated
lsim.simulate(
    60, # n steps

    'jump',  # a single word or list of words to prompt
    prompt_blur=0.7,  # reduce voltage of other cells than the prompted letter
)
push_notebook(handle=figh)  # Update the plot

In [ ]:
# another batch but with no prompt, to see how current dynamics drives the network forward
# scroll back to see the spike plot updated
lsim.simulate(
    10, # n steps
    
    '', # no prompt
)
push_notebook(handle=figh)  # Update the plot

In [ ]:
# create 2M inhibitory synapses randomly
# note: inhibitory synapses is also capped, though separately,
#       with the same MAX_SYNAPSES (defaults to 1M) number,
#       and compacted according to the same LOAD_FACTOR (defaults to 0.8)
lnet.create_inhibitory_links_randomly( 2_000_000 )

In [ ]:
# get a DataFrame about all inhibitory synapses so far
inhib_df = lnet.excitatory_synapses()
inhib_df

In [ ]:
# another batch but with no prompt, to see how current dynamics drives the network forward
# scroll back to see the spike plot updated
lsim.simulate(
    20, # n steps
    
    '', # no prompt
)
push_notebook(handle=figh)  # Update the plot

In [ ]:
# instead of updating the plot shown at the original cell batch by batch (which is handy when you batch in loops),
# you can always show all the simulated spikes, from beginning up to the moment, in any cell. 
show(lsim.fig)

In [ ]:
# another batch but with no prompt, to see how current dynamics drives the network forward
lsim.simulate(
    20, # n steps
    
    '', # no prompt
)

# however `push_notebook(handle=figh)` will have no effect once a snapshot plot has been drawn,
# you'll then have to draw new plots to see updated data.
show(lsim.fig)